In [1]:
#!/usr/bin/env python3

################################
# General Imports
################################
import csv, math, io, os, os.path, sys, random, time, json, gc, glob, statistics
from datetime import datetime
import joblib
from joblib import Parallel, delayed, dump, load

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sb

################################
# Scientific Imports
################################
import scipy
from scipy.signal import butter,filtfilt

################################
# SKLearn Imports
################################
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler

from sklearn.tree import DecisionTreeClassifier

################################
# SKTime Imports
################################
from sktime.datatypes._panel._convert import from_2d_array_to_nested, from_nested_to_2d_array, is_nested_dataframe, from_nested_to_multi_index
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.model_selection import ForecastingGridSearchCV

from sktime.classification.compose import ComposableTimeSeriesForestClassifier
from sktime.datasets import load_airline
from sktime.utils.slope_and_trend import _slope

from sktime.classification.kernel_based import Arsenal
from sktime.classification.interval_based import CanonicalIntervalForest
from sktime.classification.dictionary_based import ContractableBOSS
from sktime.classification.interval_based import DrCIF
from sktime.classification.hybrid import HIVECOTEV1
from sktime.classification.dictionary_based import IndividualBOSS
from sktime.classification.dictionary_based import IndividualTDE
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
from sktime.classification.feature_based import MatrixProfileClassifier
from sktime.classification.dictionary_based import MUSE
from sktime.classification.interval_based import RandomIntervalSpectralForest
from sktime.classification.distance_based import ShapeDTW
from sktime.classification.feature_based import SignatureClassifier
from sktime.classification.interval_based import SupervisedTimeSeriesForest
from sktime.classification.feature_based import TSFreshClassifier
from sktime.classification.dictionary_based import WEASEL

################################
# Suppress Warnings
################################
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

################################
# Initialisers
################################
default_rc_params = (16,9)
plt.rcParams["figure.figsize"] = default_rc_params
sb.set()

xNaNs = np.load("X_NAN_LIST.npy")
xTime = np.load("X_TIME_LIST.npy")

masterX = np.asarray([x[1:-1] for x in np.load("None_Or_One_Exoplanet.npy")])
masterY = np.load("None_Or_One_isplanetlist.npy")

X_nested = from_2d_array_to_nested(masterX)

################################
# Functions
################################

def ConvertDataToNestedDF(X):
    print(f"Shape of X is {X.shape}")
    x = from_2d_array_to_nested(X)
    return x

################################

In [2]:
nth_dp = 40
nth_lc = 50

minorX = np.array([x[::nth_dp] for x in masterX])[::nth_lc]
minorY = masterY[::nth_lc]

minorX_nested = from_2d_array_to_nested(minorX)

In [3]:
from sktime.classification.interval_based import DrCIF
from sktime.datasets import load_unit_test
from sklearn.metrics import f1_score

X_train, y_train = load_unit_test(split="train", return_X_y=True)
X_test, y_test = load_unit_test(split="test", return_X_y=True)
clf = DrCIF(n_estimators=10)
clf.fit(minorX_nested, minorY)

DrCIF(n_estimators=10)

In [5]:
cnv = FunctionTransformer(ConvertDataToNestedDF)

pipe = Pipeline(steps=[('nest', cnv),('algorithm',DrCIF())])

In [15]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(DrCIF(), minorX_nested, minorY, cv=2, scoring=('precision'))
cv_results

KeyboardInterrupt: 